In [1]:
import networkx as nx
import pandas as pd

import oot_graph_builder as ogb

all_actors_df = pd.read_csv("resources/VerboseOcarina/actors.csv", sep=';')
all_transition_actors_df = pd.read_csv("resources/VerboseOcarina/transition_actors.csv", sep=';')
all_spawns_df = pd.read_csv("resources/VerboseOcarina/spawns.csv", sep=';')

scenes = range(101)
setups = range(4)
cutscenes_included = (
    False,
    # True,
)

scc_1_count = 0
node_count = 0
edge_count = 0

dw_edge_count = 0
song_edge_count = 0

for scene in scenes:
    for is_cutscene in cutscenes_included:
        for setup in setups:
            G_scene = ogb.build_scene_graph(all_spawns_df,
                                            all_actors_df,
                                            all_transition_actors_df,
                                            scene,
                                            setup,
                                            is_cutscene)

            if G_scene:
                if (scc := nx.number_strongly_connected_components(G_scene)) == 1:
                    scc_1_count += 1
                else:
                    print(f"{scene = }, {setup = }, {is_cutscene = }: "
                          f"{scc} strongly connected components")

                node_count += len(G_scene.nodes)
                edge_count += len(G_scene.edges)

                spawn_count = 0

                for node, data in G_scene.nodes(data=True):
                    if data[ogb.NODE_TYPE_LABEL] == ogb.SPAWN_LABEL:
                        spawn_count += 1

                if scene not in ogb.NO_WARP_SONGS_SCENES:
                    song_edge_count += 6 * len(G_scene)

                dw_edge_count += spawn_count * len(G_scene)

print(f"There are {scc_1_count} scene setups with 1 strongly connected component each.")
print(f"There are {node_count} nodes in total.")
print(f"There are {edge_count} edges in total.")

G_union, pos_dict_norm = ogb.build_scenes(all_spawns_df,
                                          all_actors_df,
                                          all_transition_actors_df,
                                          scenes,
                                          setups,
                                          cutscenes_included,
                                          )

G_w_sw = ogb.add_save_warps(G_union)
G_w_dw = ogb.add_death_warps(G_union)
G_w_songw = ogb.add_song_warps(G_union)
G_w_sdw = ogb.add_death_warps(G_w_sw)
G_w_aw = ogb.add_song_warps(G_w_sdw)

print(f"{len(G_union) = }")
print(f"{len(G_union.edges) = }")

print(f"{len(G_w_sw) - len(G_union) = }")
print(f"{len(G_w_dw) - len(G_union) = }")
print(f"{len(G_w_songw) - len(G_union) = }")
print(f"{len(G_w_aw) - len(G_union) = }")

print(f"{len(G_w_sw.edges) - len(G_union.edges) = }")
print(f"{len(G_w_dw.edges) - len(G_union.edges) = }")
print(f"{len(G_w_songw.edges) - len(G_union.edges) = }")
print(f"{len(G_w_aw.edges) - len(G_union.edges) = }")

# Remove edges from same room; actually don't, it's two separate things and doesn't matter in a DiGraph anyway.
print(f"{dw_edge_count = }")
print(f"{song_edge_count = }")

scene = 12, setup = 0, is_cutscene = False: 6 strongly connected components
scene = 14, setup = 0, is_cutscene = False: 4 strongly connected components
scene = 62, setup = 0, is_cutscene = False: 14 strongly connected components
scene = 76, setup = 0, is_cutscene = False: 2 strongly connected components
There are 125 scene setups with 1 strongly connected component each.
There are 6764 nodes in total.
There are 231854 edges in total.
save warps: 10452
death_warps: 43314
song_warps: 35364
death_warps: 43314
song_warps: 35364
len(G_union) = 6764
len(G_union.edges) = 231854
len(G_w_sw) - len(G_union) = 0
len(G_w_dw) - len(G_union) = 0
len(G_w_songw) - len(G_union) = 0
len(G_w_aw) - len(G_union) = 0
len(G_w_sw.edges) - len(G_union.edges) = 10452
len(G_w_dw.edges) - len(G_union.edges) = 43314
len(G_w_songw.edges) - len(G_union.edges) = 35364
len(G_w_aw.edges) - len(G_union.edges) = 89130
dw_edge_count = 43314
song_edge_count = 35364
